Install PyGithub because it doesn't exist in the docker container by default.

In [ ]:
import sys
!{sys.executable} -m pip install PyGithub

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

from issue_tagging_bot.issue_data import Stage1PreprocData, Stage2PreprocData

In [3]:
stage2 = Stage2PreprocData()

In [4]:
stage2.stage1.top_n_topics()

6.topic: TeX                      48
6.topic: reproducible builds      49
6.topic: rust                     51
6.topic: xfce                     59
6.topic: steam                    68
6.topic: user experience          73
6.topic: closure-size            114
6.topic: kernel                  115
6.topic: cross-compilation       132
6.topic: gnome3                  263
6.topic: qt/kde                  345
6.topic: haskell                 606
6.topic: darwin                  610
6.topic: python                  813
6.topic: nixos                  1169
dtype: int64

In [5]:
train_set, val_set, test_set = stage2.to_datasets()

In [14]:
all_test_labels = []
all_nums = []
for issue_body, labels, num in test_set:
    all_test_labels.append(labels)
    all_nums.append(num)
    
first_all_test_labels = all_test_labels[0]
first_all_nums = all_nums[0]

first_all_test_labels, first_all_nums

(<tf.Tensor: shape=(15,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])>,
 <tf.Tensor: shape=(), dtype=int32, numpy=79304>)

In [7]:
all_test_labels_tensor = tf.reshape(tf.concat(all_test_labels, axis=0), (-1, stage2.stage1.top_n_topics().shape[0]))

In [8]:
np.sum(all_test_labels_tensor.numpy(), axis=0)

array([  4,   3,   6,  11,   3,  10,   7,  17,  22,  33,  32,  56,  60,
        85, 102])

In [10]:
all_test_labels_tensor[0]

<tf.Tensor: shape=(15,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])>

In [12]:
all_nums[0]

<tf.Tensor: shape=(), dtype=int32, numpy=79304>

In [15]:
X = tf.range(10)

In [16]:
X

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [17]:
dataset = tf.data.Dataset.from_tensor_slices(X)

In [18]:
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [25]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


In [62]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(11))
dataset = dataset.shuffle(buffer_size=10000, seed=42, reshuffle_each_iteration=False)
#dataset = dataset.shuffle(buffer_size=10000, seed=42, reshuffle_each_iteration=True)
my_x = dataset.take(3)
dataset = dataset.skip(3)
my_y = dataset.take(3)#.shuffle(buffer_size=10000, seed=42, reshuffle_each_iteration=True)
dataset = dataset.skip(3)
my_z = dataset

#my_y = my_y.shuffle(buffer_size=10000, reshuffle_each_iteration=True)
my_y = my_y.shuffle(buffer_size=10000, seed=42, reshuffle_each_iteration=True)

def print_dataset(xxx, name):
    for i, item in xxx.enumerate():
        print(f"item {i} from {name}: {item}")
    print()
        
print_dataset(my_x, "my_x")
print_dataset(my_y, "my_y")
print_dataset(my_y, "my_y")
print_dataset(my_z, "my_z")
print_dataset(my_z, "my_z")
print_dataset(my_y, "my_y")
print_dataset(my_z, "my_z")
print_dataset(my_y, "my_y")
print_dataset(my_y, "my_y")
print_dataset(my_y, "my_y")
print_dataset(my_x, "my_x")


item 0 from my_x: 1
item 1 from my_x: 5
item 2 from my_x: 6

item 0 from my_y: 8
item 1 from my_y: 7
item 2 from my_y: 10

item 0 from my_y: 8
item 1 from my_y: 7
item 2 from my_y: 10

item 0 from my_z: 9
item 1 from my_z: 4
item 2 from my_z: 3
item 3 from my_z: 2
item 4 from my_z: 0

item 0 from my_z: 9
item 1 from my_z: 4
item 2 from my_z: 3
item 3 from my_z: 2
item 4 from my_z: 0

item 0 from my_y: 8
item 1 from my_y: 7
item 2 from my_y: 10

item 0 from my_z: 9
item 1 from my_z: 4
item 2 from my_z: 3
item 3 from my_z: 2
item 4 from my_z: 0

item 0 from my_y: 7
item 1 from my_y: 8
item 2 from my_y: 10

item 0 from my_y: 7
item 1 from my_y: 8
item 2 from my_y: 10

item 0 from my_y: 7
item 1 from my_y: 10
item 2 from my_y: 8

item 0 from my_x: 1
item 1 from my_x: 5
item 2 from my_x: 6

